- **id** - id команды
- **api_id** - ключ на таблицу matches
- **long_name**	-    полное название команды
- **short_name** -	сокращённое название команды
 
 ### math

- **id** -	id матча
- **season**-	сезон
- **date** -	дата матча
- **home_team_api_id** -	api_id домашней команды, ключ на таблицу teams
- **away_team_api_id**	-api_id гостевой команды, ключ на таблицу teams
- **home_team_goals** -	количество голов домашней команды
- **away_team_goals** -	количество голов гостевой команды

ФИЛЬТРАЦИЯ ДАННЫХ

К соединённым таблицам применимы функции фильтрации данных.

Например, можно вывести id матчей, в которых команда Arsenal была гостевой.

In [ ]:
#/*Напишите запрос, который выведет полное название команды (long_name), 
#количество голов домашней команды (home_goal) и количество голов гостевой команды (away_goal) в матчах, 
#где домашней командой были команды с коротким названием 'GEN'. Отсортируйте запрос по id матча в порядке возрастания..*/
"""
SELECT 
    h.long_name long_name,
    m.home_team_goals  home_goal,
    m.away_team_goals  away_goal
FROM
    sql.matches m 
join sql.teams h  on  h.api_id = m.home_team_api_id
where 
    h.short_name = 'GEN'
ORDER BY m.id
"""

Также мы можем отфильтровать записи сразу по двум таблицам.

Например, можно оставить только записи, в которых короткое название домашней команды GEN и матчи сезона 2008/2009.

In [ ]:
"""
SELECT * /*выбор всех полей*/
FROM    
    sql.matches m /*таблица matches с алиасом m*/
    JOIN sql.teams t on t.api_id = m.home_team_api_id /*оператор соединения таблиц; таблица teams с алиасом t; условие: home_team_api_id таблицы m равен api_id таблицы t*/
WHERE
    t.short_name = 'GEN' /*столбец short_name таблицы t имеет значение GEN*/
    AND m.season = '2008/2009' /*столбец season таблицы m имеет значение 2008/2009*/
"""

In [ ]:
#/*Напишите запрос, чтобы вывести id матчей, короткое название домашней команды (home_short), 
#короткое название гостевой команды (away_short) для матчей сезона 2011/2012, 
#в которых участвовала команда с названием Liverpool. Отсортируйте по id матча в порядке возрастания.
#*/
"""
SELECT 
    m.id,
    h.short_name  home_short,
    a.short_name  away_short
FROM
    sql.matches m 
    join sql.teams h  on  h.api_id = m.home_team_api_id
    join sql.teams a  on  a.api_id = m.away_team_api_id
where 
    m.season = '2011/2012'
    and (h.long_name = 'Liverpool'  or a.long_name = 'Liverpool' )
ORDER BY m.id
"""

АГРЕГАЦИЯ ДАННЫХ

К соединённым таблицам также применимы любые агрегатные функции — самые важные функции для анализа данных.

Например, мы можем вывести сумму голов по командам для матчей, где команда выступала в гостях.

In [1]:
"""
SELECT
    t.long_name, /*столбец long_name таблицы t*/
    SUM(m.home_team_goals) + SUM(m.away_team_goals) match_goals /*функция суммирования; столбец home_team_goals таблицы m; функция суммирования; столбец away_team_goals таблицы m; новое название столбца*/
FROM
    sql.matches m /*таблица matches с алиасом m*/
    JOIN sql.teams t ON m.away_team_api_id = t.api_id /*оператор соединения таблиц; таблица teams с алиасом t; условие: away_team_api_id таблицы m равен api_id таблицы t*/
GROUP BY t.id /*группировка по столбцу id таблицы t*/
"""

'\nSELECT\n    t.long_name, /*столбец long_name таблицы t*/\n    SUM(m.home_team_goals) + SUM(m.away_team_goals) match_goals /*функция суммирования; столбец home_team_goals таблицы m; функция суммирования; столбец away_team_goals таблицы m; новое название столбца*/\nFROM\n    sql.matches m /*таблица matches с алиасом m*/\n    JOIN sql.teams t ON m.away_team_api_id = t.api_id /*оператор соединения таблиц; таблица teams с алиасом t; условие: away_team_api_id таблицы m равен api_id таблицы t*/\nGROUP BY t.id /*группировка по столбцу id таблицы t*/\n'

Обратите внимание! В данном запросе была использована группировка по столбцу id таблицы teams, хотя этот столбец не выводится в запросе. Это необходимо для того, чтобы команды с одинаковым названием, если такие найдутся, не группировались между собой. Группировка по названию команды в данном запросе будет неверной, так как есть несколько команд с одинаковым полным названием — мы говорили об этом в начале модуля.

Также, применяя агрегатные функции к соединённым таблицам, обращайте внимание на указание алиасов (или таблиц) при группировке и указании столбцов агрегатных функций. В нашей соединённой таблице есть два столбца с названием id, и если бы мы сформировали запрос без указания таблицы, как указано ниже, то...

Мы можем использовать оператор HAVING для фильтрации сгруппированных данных.

Поставим задачу — вывести таблицу с суммарным количеством забитых голов в матчах по командам и сезонам для команд, в которых суммарное количество голов в матчах сезона больше 100.

In [ ]:
"""
SELECT
    m.season, /*столбец season таблицы m*/
    t.long_name, /*столбец long_name таблицы t*/
    SUM(m.home_team_goals) + SUM(m.away_team_goals) total_goals /*функция суммирования; столбец home_team_goals таблицы m; функция суммирования; столбец away_team_goals таблицы m; новое название столбца*/
FROM sql.matches m /*таблица matches с алиасом m*/
JOIN sql.teams t ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id /*оператор соединения таблиц; таблица teams с алиасом t; условие: home_team_api_id таблицы m равен api_id таблицы t или away_team_api_id таблицы m равен api_id таблицы t*/
GROUP BY m.season, t.id /*группировка по столбцам season таблицы m и id таблицы t*/
HAVING SUM(m.home_team_goals) + SUM(m.away_team_goals) > 100 /*оператор фильтрации сгруппированных данных; функция суммирования; home_team_goals таблицы m; функция суммирования; away_team_goals таблицы m; больше 100*/
"""

In [ ]:
#/*Напишите запрос, с помощью которого можно вывести список полных названий команд, 
#сыгравших в гостях 150 и более матчей. Отсортируйте список по названию команды.*/
"""
SELECT
    t.long_name
FROM sql.matches m
JOIN sql.teams t ON t.api_id = m.away_team_api_id
GROUP BY t.id
HAVING count(away_team_api_id) >= 150
"""